In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# syft absolute
import syft as sy
from syft import test_settings

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080",
    email="data_scientist@openmined.org",
    password="verysecurepassword",
)

In [ ]:
high_client.custom_api.api_endpoints()

In [ ]:
high_client.api.services.bigquery.test_query

In [ ]:
dataset_1 = test_settings.get("dataset_1", default="dataset_1")
dataset_2 = test_settings.get("dataset_2", default="dataset_2")
table_1 = test_settings.get("table_1", default="table_1")
table_2 = test_settings.get("table_2", default="table_2")
table_2_col_id = test_settings.get("table_2_col_id", default="table_id")
table_2_col_score = test_settings.get("table_2_col_score", default="colname")
query_limit_size = test_settings.get("query_limit_size", default=10000)

In [ ]:
FUNC_NAME = "popular"
QUERY = f"SELECT {table_2_col_id}, AVG({table_2_col_score}) AS average_score \
    FROM {dataset_2}.{table_2} \
    GROUP BY {table_2_col_id} \
    LIMIT {query_limit_size}"

result = high_client.api.services.bigquery.test_query(sql_query=QUERY)

In [ ]:
assert len(result) == query_limit_size

In [ ]:
res1 = high_client.api.services.bigquery.submit_query(func_name=FUNC_NAME, query=QUERY)
res1

In [ ]:
def extract_code_path(response):
    # stdlib
    import re

    pattern = r"client\.code\.(\w+)\(\)"
    match = re.search(pattern, str(response))
    if match:
        extracted_code = match.group(1)
        return extracted_code
    return None

In [ ]:
func_name = extract_code_path(res1)
print(func_name)

In [ ]:
assert "popular" in func_name

In [ ]:
api_method = getattr(high_client.code, func_name, None)
api_method

In [ ]:
with sy.raises(
    sy.SyftException(public_message="*Your code is waiting for approval*"), show=True
):
    result = api_method()

In [ ]:
FUNC_NAME = "large_sample"
LARGE_SAMPLE_QUERY = f"SELECT * FROM {dataset_2}.{table_2} LIMIT {query_limit_size}"

In [ ]:
res2 = high_client.api.services.bigquery.submit_query(
    func_name=FUNC_NAME, query=LARGE_SAMPLE_QUERY
)

In [ ]:
func_name = extract_code_path(res2)
print(func_name)

In [ ]:
assert "large_sample" in func_name

In [ ]:
api_method_2 = getattr(high_client.code, func_name, None)
api_method_2

In [ ]:
with sy.raises(
    sy.SyftException(public_message="*Your code is waiting for approval*"), show=True
):
    result = api_method_2()

In [ ]:
server.land()